# Experiments

### Setup

In [2]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv()

True

Here is the RAG Application that we've been working with throughout this course

In [3]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [5]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "deez-nuts"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

/Users/aditya/Desktop/MAT496_langsmith/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-94dde510' at:
https://smith.langchain.com/o/6176107e-37d8-4611-acb8-9f68bf99512f/datasets/cb90b49f-cd3c-41ba-bc84-e4e3a60df060/compare?selectedSessions=3ebf2504-e190-49c3-af9b-d72c777d9171




10it [00:27,  2.71s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.990685,0664f2fa-6d2a-483e-a1f0-82de1dbf5bdf,e253de4c-2bda-4c5e-8113-6a3a3f23805b
1,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1,2.374509,192ed178-0707-48b9-9845-3fe2f53d77ac,5b5555d2-b237-4ef5-889d-595672322c7f
2,Does LangSmith support offline evaluation?,The context does not mention offline evaluatio...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.654779,368ed475-0a2b-41aa-b4cf-4b1043694d5c,05c70be3-c86d-4e88-89f0-f8c5879864c9
3,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,2.367556,3c28f838-be16-4a4f-b70c-e311c6429589,f930d395-43cb-4281-83bd-fe6b5782c7ec
4,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for LLM observ...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.499658,5d6ca0bf-20a9-471e-88b3-05b2f0895852,9dadd3af-9f95-422d-bf5a-09dd52710cc7
5,How do I pass metadata in with @traceable?,To pass metadata in with `@traceable` in LangS...,None,You can pass metadata with the @traceable deco...,1,2.314328,651bf774-0e9f-4179-91fc-2abdef7278b5,82b1dcae-5d09-4769-bd6b-734b7ae81c76
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,3.253066,72aabf56-7dc9-45b1-883c-630f82aad67e,bdb4e1ec-bf98-442c-adc3-b0f630107780
7,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,2.795228,b3aeb7ae-0ee3-4fd8-9634-5436e7beaaa3,21aa08f1-c0dd-4b99-8389-1234e89441a5
8,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,3.352034,e2084c66-8c3a-494f-b853-561a48f20f1d,e57c724b-ff4d-48c9-98d2-57422212c234
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.996191,f0df5aaf-49a0-437a-87e4-7d5ab49f9f25,1b343ee6-0727-4d80-886c-b096b88bb13b


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [6]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-ac16ed7c' at:
https://smith.langchain.com/o/6176107e-37d8-4611-acb8-9f68bf99512f/datasets/cb90b49f-cd3c-41ba-bc84-e4e3a60df060/compare?selectedSessions=db6b9422-4af0-49ea-b8d6-7e15ae82082f




10it [00:27,  2.74s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.189943,0664f2fa-6d2a-483e-a1f0-82de1dbf5bdf,12891cb5-6cab-4ee8-802f-69df471021fb
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,3.341816,192ed178-0707-48b9-9845-3fe2f53d77ac,18e16851-1c7c-437e-9bbe-23d53019e49c
2,Does LangSmith support offline evaluation?,The provided context does not mention support ...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.099428,368ed475-0a2b-41aa-b4cf-4b1043694d5c,9d4364de-e0ac-48e0-9fc2-4c20fabace13
3,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.835827,3c28f838-be16-4a4f-b70c-e311c6429589,2123cacb-ea55-453c-858e-510f9c7dd2ef
4,Can LangSmith be used for finetuning and model...,"LangSmith is designed for LLM observability, e...",None,"Yes, LangSmith can be used for fine-tuning and...",1,3.847204,5d6ca0bf-20a9-471e-88b3-05b2f0895852,cf977f76-5407-4e76-98fe-2f9ef012300a
5,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you need t...",None,You can pass metadata with the @traceable deco...,1,2.761250,651bf774-0e9f-4179-91fc-2abdef7278b5,43928c72-863a-441d-832f-407979c9d10f
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.511885,72aabf56-7dc9-45b1-883c-630f82aad67e,3c12aee5-ae5f-460d-9346-2e2880a8589c
7,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,2.913665,b3aeb7ae-0ee3-4fd8-9634-5436e7beaaa3,4575265d-82cc-4047-b82f-e286136bdaa1
8,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,3.214412,e2084c66-8c3a-494f-b853-561a48f20f1d,e04d4044-492c-43ea-91f9-80c471bc0942
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.020619,f0df5aaf-49a0-437a-87e4-7d5ab49f9f25,1680d36b-0ef9-40a4-a9c8-1e30d62f55db


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [ ]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

In [14]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "e2084c66-8c3a-494f-b853-561a48f20f1d",
            "3c28f838-be16-4a4f-b70c-e311c6429589"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-962f30e9' at:
https://smith.langchain.com/o/6176107e-37d8-4611-acb8-9f68bf99512f/datasets/cb90b49f-cd3c-41ba-bc84-e4e3a60df060/compare?selectedSessions=e834a79a-d17e-464e-9245-d4feb6998e4e




2it [00:04,  2.35s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.992878,3c28f838-be16-4a4f-b70c-e311c6429589,06a098c8-d8f4-4acb-8a86-81e691d33619
1,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,2.127572,e2084c66-8c3a-494f-b853-561a48f20f1d,edf135a1-7b03-4373-96e5-b08ec26fb3ec


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [13]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-6709d1ea' at:
https://smith.langchain.com/o/6176107e-37d8-4611-acb8-9f68bf99512f/datasets/cb90b49f-cd3c-41ba-bc84-e4e3a60df060/compare?selectedSessions=8248d90e-81ce-46c9-8b30-6aa9b78cef31




20it [00:50,  2.52s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.974482,0664f2fa-6d2a-483e-a1f0-82de1dbf5bdf,100a8e90-711a-4d80-93a6-659467ac6842
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.549105,192ed178-0707-48b9-9845-3fe2f53d77ac,f460f319-0767-4c0a-a9f1-f46e09edb3c8
2,Does LangSmith support offline evaluation?,The provided context only mentions online eval...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.809238,368ed475-0a2b-41aa-b4cf-4b1043694d5c,a1b68f62-3b9d-40eb-8308-569f790e01b7
3,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,2.678483,3c28f838-be16-4a4f-b70c-e311c6429589,67497f05-b863-4b44-9a91-8ed6662ec364
4,Can LangSmith be used for finetuning and model...,LangSmith is not designed for fine-tuning or m...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.843881,5d6ca0bf-20a9-471e-88b3-05b2f0895852,3e999dcf-2829-403d-a2df-1907a48ea609
5,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you can...",None,You can pass metadata with the @traceable deco...,1,2.451814,651bf774-0e9f-4179-91fc-2abdef7278b5,bad4a63e-7d08-483b-bc98-48ec3cf2a353
6,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.865634,72aabf56-7dc9-45b1-883c-630f82aad67e,94951122-f68a-4692-ab2e-1a70bd54a5ca
7,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,3.131554,b3aeb7ae-0ee3-4fd8-9634-5436e7beaaa3,c32525d7-2ec7-49a4-836c-9bda6b4ce6b6
8,How can I trace with the @traceable decorator?,To trace with the @traceable decorator using L...,None,To trace with the @traceable decorator in Pyth...,1,3.780284,e2084c66-8c3a-494f-b853-561a48f20f1d,f0f66875-ba93-46a2-b2e6-5dee010077d2
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.432201,f0df5aaf-49a0-437a-87e4-7d5ab49f9f25,d2cbbe29-cc4d-44d0-aa72-241debdcd92a


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [10]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-945bb80f' at:
https://smith.langchain.com/o/6176107e-37d8-4611-acb8-9f68bf99512f/datasets/cb90b49f-cd3c-41ba-bc84-e4e3a60df060/compare?selectedSessions=b7755d6b-0661-4887-a752-d7d1892783a3




10it [00:59,  5.95s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,The provided context does not mention support ...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.115516,368ed475-0a2b-41aa-b4cf-4b1043694d5c,8cea4bb8-56e3-478d-946d-2bbfb653607c
1,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1,2.361447,192ed178-0707-48b9-9845-3fe2f53d77ac,93be1461-8929-4bf9-bf77-73e894fdabed
2,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,1.862816,3c28f838-be16-4a4f-b70c-e311c6429589,692b5d84-8739-433a-b9cf-03167000dd96
3,Can LangSmith be used for finetuning and model...,LangSmith is primarily a platform for building...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.391587,5d6ca0bf-20a9-471e-88b3-05b2f0895852,fb395439-dcf6-4a8d-802e-0e212893e7ab
4,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can us...",None,You can pass metadata with the @traceable deco...,1,2.476153,651bf774-0e9f-4179-91fc-2abdef7278b5,76e07328-fbed-4695-b9e0-8223e17b6955
5,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.110479,72aabf56-7dc9-45b1-883c-630f82aad67e,422465ad-3c78-44e6-9379-a6dbb13a82be
6,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,3.112809,b3aeb7ae-0ee3-4fd8-9634-5436e7beaaa3,84b0ccd3-4646-4d50-92a7-f1f1cb6baa21
7,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,2.996796,e2084c66-8c3a-494f-b853-561a48f20f1d,98fe53ea-263c-41ed-9e36-77a604c879d5
8,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.778701,f0df5aaf-49a0-437a-87e4-7d5ab49f9f25,3f1aa459-f43a-4b89-84fe-dad7f1a2bc5a
9,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,58.980421,0664f2fa-6d2a-483e-a1f0-82de1dbf5bdf,c17b0d46-f3ae-40ce-aeb9-af37bb720d58


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [11]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-49c275b1' at:
https://smith.langchain.com/o/6176107e-37d8-4611-acb8-9f68bf99512f/datasets/cb90b49f-cd3c-41ba-bc84-e4e3a60df060/compare?selectedSessions=439b718b-05df-4b9a-8b27-ca5eeed6931c




10it [02:23, 14.39s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.970036,0664f2fa-6d2a-483e-a1f0-82de1dbf5bdf,af82b3b7-c2fd-4dfe-918d-6476b0a2f67e
1,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1,2.548548,192ed178-0707-48b9-9845-3fe2f53d77ac,7a07b57a-c57c-49f6-aea0-e2554edbebc6
2,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.724230,368ed475-0a2b-41aa-b4cf-4b1043694d5c,0734a285-df8a-48e8-a3f0-fd5a164db320
3,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,2.423431,3c28f838-be16-4a4f-b70c-e311c6429589,1c12ddb6-ec52-47fa-a280-ee07d8e8f51d
4,Can LangSmith be used for finetuning and model...,LangSmith is designed for LLM observability an...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.095292,5d6ca0bf-20a9-471e-88b3-05b2f0895852,61e32baa-752d-4b2a-9ee7-a6cb1d250370
5,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` decorat...,None,You can pass metadata with the @traceable deco...,1,2.541891,651bf774-0e9f-4179-91fc-2abdef7278b5,5bda9713-b002-4a9f-8d9e-ce4aee093fe2
6,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.686949,72aabf56-7dc9-45b1-883c-630f82aad67e,268403ee-cc68-43cd-a51d-8404d57e6a78
7,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,To set up tracing to LangSmith while using Lan...,0,2.229662,b3aeb7ae-0ee3-4fd8-9634-5436e7beaaa3,896a2fa4-e31f-4bfb-a442-ebe990fe3714
8,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,2.268109,e2084c66-8c3a-494f-b853-561a48f20f1d,d4f72066-1756-43c7-9958-5d1bbc13ba2c
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,123.778899,f0df5aaf-49a0-437a-87e4-7d5ab49f9f25,e98efc7b-df7e-41ae-b921-5e2458fbfad9
